In [ ]:
words = []
b_dict = {}

with open('input.txt', 'r', encoding='utf-8') as f:
    words = f.read().splitlines()

#input_text = "how are you today everyone? Are you feeling good? Are you ok? Are you fine? how are you doing? Are there anyone we missed? are"
#words = input_text.lower().split()

for i in range(len(words) - 1):
    bigram = (words[i], words[i + 1])
    b_dict[bigram] = b_dict.get(bigram, 0) + 1


b_dict = sorted(b_dict.items(), key = lambda kv: -kv[1]) # sort by the count (value)
#b_dict

In [ ]:
# assign numbers to words. In other words encode them.
vocab = sorted(list(set(words)))
print(len(vocab))

wtoi = {}
itow = {}

wtoi = {w:i+1 for i,w in enumerate(vocab)}
wtoi['.'] = 0
itow = {i:w for w,i in wtoi.items()}

#print(len(wtoi.items()))
#print(sorted(wtoi.items(), key = lambda i: i[1]))
#print(sorted(itow.items(), key = lambda i: i[0]))

In [ ]:
# numbers - > vectors (one hot encoding)
import torch

vocab_size = len(vocab) + 1 # +1 for the special cha '.'
vocab_encodings = torch.zeros(vocab_size, vocab_size) # 1 x 9

print (vocab_encodings.shape)
for i in range(len(b_dict)):
  biagram = b_dict[i][0]
  count = b_dict[i][1]
  if count > 0:
    vocab_encodings[wtoi[biagram[0]], wtoi[biagram[1]]] = count

#print(vocab_encodings)

In [ ]:
# count -> prob distirbution
P = vocab_encodings.float()
P /= P.sum(1, keepdim=True)
P = P[1:]
P

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(28,28))
plt.imshow(vocab_encodings, cmap='Blues')
for i in range(15):
    for j in range(15):
        chstr = itow[i] + " " + itow[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, vocab_encodings[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [ ]:
g = torch.Generator().manual_seed(123456)

for i in range(3):

    out = []
    ix = 0
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itow[ix] + " ")


        if itow[ix].endswith("?"):
          break

    print(''.join(out))